In [3]:
import matplotlib.pyplot as plt
import numpy as np
import math
from MultiClassCSBM import MultiClassCSBM
from CSBMhet import CSBMhet
from CSBMhom import CSBMhom
from metrics import mmd_rbf, total_variation_distance

import torch
from sklearn.manifold import TSNE
from torch_geometric.nn import Node2Vec

In [4]:
dimensions = 40
gamma = 2 * dimensions
n = 200
classes = 8

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
time_steps = []
csbm = CSBMhom(n=n, classes=classes, dimensions=dimensions)
edge_index_0 = torch.tensor([csbm.edge_sources, csbm.edge_targets], dtype=torch.long)
edge_index_0

tensor([[  0,   0,   0,  ..., 199, 199, 199],
        [ 20,  23,  32,  ..., 185, 189, 194]])